In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

#'MeanGradient', 'MaxPoint', 'DistRatio', 'PreviousChange','MeanGOther', 'PChangeOther', 'MaxPointOther', 'DistRatOther'
PreX_train = [pd.read_csv("ML2014.csv"), pd.read_csv("ML2016.csv"), pd.read_csv("ML2015.csv"), 
              pd.read_csv("ML2018.csv"), pd.read_csv("ML2019.csv")]
PreX_test = pd.read_csv("ML2017.csv")

X_test = PreX_test.dropna()
y_test = X_test['Type'].replace(2, 1, regex=True)
X_test = X_test[['MeanGradient','MaxPoint',     'PreviousChange','DistRatio',
                 'MeanGOther',  'MaxPointOther','PChangeOther',  'DistRatOther']]

In [51]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from  sklearn.preprocessing import *
from sklearn.linear_model import*

# Round to the nearest number function
def myround(x, base):
    return base * round(x/base)

importance = np.array([0, 0, 0, 0, 0, 0, 0, 0]) #Importance of features in decision tree
index_sc = set()
sc_year = {}
main_sc = {}
value = 1

for data in PreX_train:
    X_train = data.dropna()
    y_train = X_train['Type'].replace(2, 1, regex=True)
    X_train = X_train[['MeanGradient','MaxPoint',     'PreviousChange','DistRatio',
                       'MeanGOther',  'MaxPointOther','PChangeOther',  'DistRatOther']]
    
    for number in range(0, 10):
        
        #Run Decision Tree 
        clfD = DecisionTreeClassifier(class_weight = {1: 20}, max_depth=70).fit(X_train, y_train)        
        importance = importance + clfD.feature_importances_        
        l = clfD.predict(X_test)
        temp = X_test[l==value].index
        
        #Add all time indexes rounded off into a set
        index_sc.clear()
        for time_index in temp:        
            index_sc.add(myround(time_index, 60))        
        
        #Increment dictionary value if time is predicted as a SC
        for time_index in index_sc:
            sc_year[time_index] = sc_year.get(time_index, 0) + 1
            
    #If time_index appears in all predictions for on year run add it to the main_sc dictionary
    for time_index in sc_year:
        if sc_year[time_index]==10:
            main_sc[time_index] = main_sc.get(time_index, 0) + 1            
            
    sc_year.clear()

# If time_index appears in one year ot more add it to the final prediction 
main = []
for time_index in main_sc:
    if main_sc[time_index]>=2:
        main.append(time_index)
        
#List all features in order of importance
importance = list(importance/50)
ind = [importance.index(i) for i in sorted(importance, reverse=True)]
for i in ind:
    print(X_train.columns[i], ': ', importance[i])

MeanGOther :  0.7219861206462571
PChangeOther :  0.15511917261267122
PreviousChange :  0.03612527223559169
MeanGradient :  0.02847456188341243
DistRatOther :  0.020780621002728393
DistRatio :  0.018114847312723236
MaxPointOther :  0.01321927186014093
MaxPoint :  0.006180132446474817


In [52]:
#This shows the actual SC occurnces for a year and shows what my decision tree predicted.

main = sorted(main)
print("Predicted:")
print(main)
print(len(main))
print("")

temp = PreX_test[PreX_test["Type"]>=1].dropna().index
index_sc = set()
index_sc.clear()
for time_index in temp:        
    index_sc.add(myround(time_index, 60))   
    
index_sc = sorted(index_sc)
print("Actual:")
print(index_sc)
print(len(index_sc))
print("")

common_elements = list(set(index_sc) & set(main))
print("Common:")
print(sorted(common_elements))
print(len(common_elements))
print("")

print("Recall: ", len(common_elements) / len(index_sc))
print("Precision: ", len(common_elements) / len(main))

#PreX_test[(PreX_test["Type"]>=1)].iloc[0:60].dropna()

Predicted:
[36480, 211200, 248880, 262740, 272160, 282600, 283440, 346620, 348840, 349920, 358560, 359940, 364320, 366960, 368700, 369300]
16

Actual:
[36480, 159300, 178380, 193500, 211200, 232740, 263700, 272160, 282600, 348840, 358560, 359940, 366960, 369300, 422280]
15

Common:
[36480, 211200, 272160, 282600, 348840, 358560, 359940, 366960, 369300]
9

Recall:  0.6
Precision:  0.5625


In [56]:
# suu = 156980	                      	                  
# maxp = PreX_test["MaxPoint"].iloc[suu]
# width = 100
# xpoints = np.array(PreX_test["DateTime"].iloc[suu-width:suu+width+maxp])
# ypoints = np.array(PreX_test["OtherH"] .iloc[suu-width:suu+width+maxp])

# plt.plot(xpoints, ypoints, label="OtherH")

# plt.plot(xpoints[width], ypoints[width], marker="o", markersize=5)
# plt.plot(xpoints[width+maxp], ypoints[width+maxp], marker="o", markersize=5)
# plt.xticks([0, width, width*2])
# plt.yscale('linear')
# plt.legend(loc="upper left")
# plt.show()

# #xpoints1 = np.array(df["BzE"].iloc[suu-width:suu+width+maxp].index)
# #ypoints1 = np.array(df["BzE"].iloc[suu-width:suu+width+maxp])
# #xpoints2 = np.array(df["Pres"].iloc[suu-width:suu+width+maxp].index)
# #ypoints2 = np.array(df["Pres"].iloc[suu-width:suu+width+maxp])
# #plt.plot(xpoints1, ypoints1,  label="IMF BzE")
# #plt.plot(xpoints2, ypoints2,  label="Pressure")
# #plt.loglog(xpoints1[width], ypoints1[width], marker="o", markersize=5)
# #plt.loglog(xpoints2[width], ypoints2[width], marker="o", markersize=5)

In [55]:
# X_train = PreX_train[0]
# X_train = X_train.dropna()
# y_train = X_train['Type'].replace(2, 1, regex=True)
# X_train = X_train[['MeanGradient', 'MaxPoint', 'PreviousChange',
#                    'MeanGOther', 'PChangeOther', 'MaxPointOther', 'DistRatOther', 'DistRatio']]
# X_test = pd.read_csv("ML2014.csv")
# X_test = X_test.dropna()
# y_test = X_test['Type'].replace(2, 1, regex=True)
# X_test = X_test[['MeanGradient', 'MaxPoint', 'PreviousChange',
#                  'MeanGOther', 'PChangeOther', 'MaxPointOther', 'DistRatOther', 'DistRatio']]
# importance = np.array([0, 0, 0, 0, 0, 0, 0, 0])
# for i in range(0,50):
#     clfD = DecisionTreeClassifier(class_weight = 'balanced', max_depth=30).fit(X_train, y_train)
#     importance = importance + clfD.feature_importances_
# importance = list(importance/50)

# ind = [importance.index(i) for i in sorted(importance, reverse=True)]
# for i in ind:
#     print(X_train.columns[i], ': ', importance[i])

PChangeOther :  0.37616510955881516
MeanGOther :  0.20805968374414788
dbn_nez :  0.11248257088661856
PreviousChange :  0.11067556027145951
MeanGradient :  0.09587584839792983
MaxPointOther :  0.04873635582105554
MaxPoint :  0.04800487131997364

precision {'class_weight': {1: 20}, 'max_depth': 70}  {'class_weight': 'balanced', 'max_depth': 30}
recall {'class_weight': 'balanced', 'max_depth': 10}

BIG:
class_weight = 'balanced', max_depth=30:
MeanGOther :  0.9872417649205167
MeanGradient :  0.006686358207849944
PChangeOther :  0.003269756731754063
PreviousChange :  0.001939027058299882
MaxPointOther :  0.00044061760915292404
dbn_nez :  0.00037961151778053424
MaxPoint :  4.28639546459116e-05

precision {'class_weight': 'balanced', 'max_depth': 30}

In [54]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# X_train = PreX_train[0]
# X_train = X_train.dropna()
# y_train = X_train['Type'].replace(2, 1, regex=True)
# X_train = X_train[['MeanGradient', 'MaxPoint', 'PreviousChange',
#                    'MeanGOther', 'PChangeOther', 'MaxPointOther']]
# X_test = pd.read_csv("ML2014.csv")
# X_test = PreX_test.dropna()
# y_test = X_test['Type'].replace(2, 1, regex=True)
# X_test = X_test[['MeanGradient', 'MaxPoint', 'PreviousChange',
#                  'MeanGOther', 'PChangeOther', 'MaxPointOther']]

# clf = DecisionTreeClassifier()
# grid_params = {'class_weight':[None,'balanced',{1:2},{1:3},{1:4},{1:5},{1:10},{1:20},{1:50}],
#                'max_depth':[None]+list(np.linspace(10, 100, 10, dtype = int))}
# grid_ans = GridSearchCV(clf, param_grid = grid_params , scoring = 'precision')
# grid_ans.fit(X_train, y_train)
# y_scores = grid_ans.predict(X_test)
# print(grid_ans.best_params_)
# print(grid_ans.best_score_)    
# print(recall_score(y_test, y_scores), " ", precision_score(y_test, y_scores))